**Задание 1**  

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [29]:
#Вариант Константина Башевого

df_2 = df.groupby('userId').agg({'timestamp': ['min', 'max', 'count']}).reset_index()
df_2_100 = df_2[df_2['count'] > 100.0]
df_2_100['time_life'] = df_2_100['max'] - df_2_100['min']
df_2_100.head()

KeyError: 'count'

In [27]:
df_2['time_life'] = df_2['max'] - df_2['min']
df_2.head()

KeyError: 'max'

In [21]:
# оставляем уникальных пользователей и даты
df_time = df.groupby('userId').agg(['min', 'max'])['timestamp'].reset_index()
df_time['time_life'] = df_time['max'] - df_time['min']
df_time = df_time[['userId', 'time_life']]
df_time.head()

,userId,time_life
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101


In [10]:
# оставляем пользователей, выставивших более 100 оценок
df_group = df.groupby('userId').count()
df_user_100 = df_group[df_group['rating'] > 100.0].reset_index()
#df_user_100 = df_more_100[['userId']] # оставляем один столбец
df_user_100.head()

,userId,movieId,rating,timestamp
0,4,204,204,204
1,8,116,116,116
2,15,1700,1700,1700
3,17,363,363,363
4,19,423,423,423


In [23]:
df_user_life = df_user_100.merge(df_time, how='left', left_on=['userId'], right_on=['userId'])
df_user_life.head()

,userId,time_life
0,4,203560
1,8,85187
2,15,471393496
3,17,8053
4,19,5282


**Задание  2**  

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [24]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [25]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [26]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [27]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [29]:
merge1 = client_base.merge(rzd, how='outer')
merge1

,client_id,address,rzd_revenue
0,111,Комсомольская 4,1093.0
1,112,Энтузиастов 8а,2810.0
2,113,Левобережная 1а,10283.0
3,114,Мира 14,5774.0
4,115,ЗЖБИиДК 1,981.0
5,116,Строителей 18,NaN
6,117,Панфиловская 33,NaN
7,118,Мастеркова 4,NaN


In [30]:
merge2 = merge1.merge(auto, how='outer')
merge2

,client_id,address,rzd_revenue,auto_revenue
0,111,Комсомольская 4,1093.0,NaN
1,112,Энтузиастов 8а,2810.0,NaN
2,113,Левобережная 1а,10283.0,57483.0
3,114,Мира 14,5774.0,83.0
4,115,ЗЖБИиДК 1,981.0,912.0
5,116,Строителей 18,NaN,4834.0
6,117,Панфиловская 33,NaN,98.0
7,118,Мастеркова 4,NaN,NaN


In [32]:
merge3 = merge2.merge(air, how='outer')
otvet1 = merge3
otvet1

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [34]:
otvet2 = otvet1[['client_id', 'rzd_revenue', 'auto_revenue', 'air_revenue']]
otvet2

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


**Задание 3**  

3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


4. скорее всего с координатами было передано дата-время. Можно соединить таблицы по Пользователю и Дате-времени
   если даты нет, но посчитать средние координаты и объединить таблицы по пользователю.

5. посчитается середина маршрута пользователя.
   
6. по координатам можно определить Место нахождения пользователя и маршрут (если есть время,еще и скорость)